In [1]:
import httplib2
from apiclient import errors
from apiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow
import os
from dotenv import load_dotenv
project_folder = os.path.expanduser('/Users/oleksandrdagayev/code/jupyter_SEO/')  # adjust as appropriate
load_dotenv(os.path.join(project_folder, '.env')) #create a text file called .env and paste  
import numpy as np
import pandas as pd
import json
from collections import defaultdict as dd

In [2]:
# Redirect URI for installed apps
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

CLIENT_ID = os.getenv('CLIENT_ID') #get this from setting up a Google App see here 
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

OAUTH_SCOPE = 'https://www.googleapis.com/auth/webmasters.readonly'

# Run through the OAuth flow and retrieve credentials
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print('Go to the following link in your browser: ' + authorize_url) #this will generate a link - make sure to click the link

__init__() takes at most 4 positional arguments (5 given)


Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=1098023451168-irepb3tq7bbghm7aak97tcui6gpqjfen.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&access_type=offline&response_type=code


In [ ]:
#the above gets you to the credential setp.

In [3]:
code = "4/vgE8_aYjC6f3robIB6Z-X6ZYK2MxeRQo0jN-AhJwQ9GckoaeN_lBzFI" #this was my authcode used with my instance - you will need to use your own
credentials = flow.step2_exchange(code)

# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()
http = credentials.authorize(http)

In [4]:
webmaster_search_analytics = build('webmasters', 'v3', http=http) #build the service

In [5]:
targetdomain = 'https://amino.com' #specify your domain here. Make sure to include protocol and if domain is www. or not. 

In [8]:
request = {
    'startDate': '2020-01-11', #all requests must include a start AND end date
    'endDate': '2020-01-12',
    'dimensions': ['date','query','page'], #select the dimensions of your response
    #'dimensionsFilterGroups':[{'filters'}:[{'dimension':'device','expression':'mobile'}]] - this is sample filter
    'rowLimit':25000, #this is the maximum row limit for GSC export data
    'startRow':0
      }

In [9]:
test_response = webmaster_search_analytics.searchanalytics().query(siteUrl=targetdomain,body=request).execute()

In [10]:
test_response

{'rows': [{'keys': ['2020-01-11',
    'hysterectomy',
    'https://amino.com/blog/laparoscopic-hysterectomy-cost/'],
   'clicks': 13.0,
   'impressions': 283.0,
   'ctr': 0.045936395759717315,
   'position': 1.0},
  {'keys': ['2020-01-12',
    'hysterectomy',
    'https://amino.com/blog/laparoscopic-hysterectomy-cost/'],
   'clicks': 7.0,
   'impressions': 232.0,
   'ctr': 0.03017241379310345,
   'position': 1.3577586206896552},
  {'keys': ['2020-01-11',
    'dr james hunderup',
    'https://amino.com/dr-james-hunderup/'],
   'clicks': 2.0,
   'impressions': 5.0,
   'ctr': 0.4,
   'position': 16.2},
  {'keys': ['2020-01-11',
    'dr renee ewing',
    'https://amino.com/dr-renee-ewing/'],
   'clicks': 2.0,
   'impressions': 3.0,
   'ctr': 0.6666666666666666,
   'position': 16.666666666666664},
  {'keys': ['2020-01-11',
    'dr steven barna tampa fl',
    'https://amino.com/dr-steven-barna/'],
   'clicks': 2.0,
   'impressions': 2.0,
   'ctr': 1.0,
   'position': 13.0},
  {'keys': ['2020

In [11]:
test_data = {'date':[],'searchQuery':[],'page':[], 'clicks':[],'impressions':[],'ctr':[], 'position':[]}
    
for i in test_response['rows']:
    date = i['keys'][0] #assumes that date is first
    test_data['date'].append(date)
    searchQuery = i['keys'][1] #assumes that query is second
    test_data['searchQuery'].append(searchQuery)
    page = i['keys'][2] #assumes that query is second
    test_data['page'].append(page)
    clicks = i['clicks']
    test_data['clicks'].append(clicks)
    impressions = i['impressions']
    test_data['impressions'].append(impressions)
    ctr = i['ctr']
    test_data['ctr'].append(ctr)
    position = i['position']
    test_data['position'].append(position)

In [12]:
pd.DataFrame(test_data) #this is meant to validate that everything you're doing is correct so far

,date,searchQuery,page,clicks,impressions,ctr,position
0,2020-01-11,hysterectomy,https://amino.com/blog/laparoscopic-hysterecto...,13.0,283.0,0.045936,1.000000
1,2020-01-12,hysterectomy,https://amino.com/blog/laparoscopic-hysterecto...,7.0,232.0,0.030172,1.357759
2,2020-01-11,dr james hunderup,https://amino.com/dr-james-hunderup/,2.0,5.0,0.400000,16.200000
3,2020-01-11,dr renee ewing,https://amino.com/dr-renee-ewing/,2.0,3.0,0.666667,16.666667
4,2020-01-11,dr steven barna tampa fl,https://amino.com/dr-steven-barna/,2.0,2.0,1.000000,13.000000
...,...,...,...,...,...,...,...
24995,2020-01-11,eric tallarico md,https://amino.com/dr-eric-tallarico/,0.0,2.0,0.000000,6.500000
24996,2020-01-11,eric tatar,https://amino.com/dr-eric-tatar/,0.0,1.0,0.000000,10.000000
24997,2020-01-11,eric toloza md,https://amino.com/dr-eric-toloza-md-phd/,0.0,1.0,0.000000,12.000000
24998,2020-01-11,eric varboncouer md,https://amino.com/dr-eric-varboncouer-md/,0.0,1.0,0.000000,13.000000


In [13]:
def get_all_response(domain,request): #this is a function that agreggates responses for API calls over 25000 rows
    data = {'date':[],'searchQuery':[],'page':[], 'clicks':[],'impressions':[],'ctr':[], 'position':[]}
    startRow = 0
    request['startRow'] = startRow
    initresponse = webmaster_search_analytics.searchanalytics().query(siteUrl=targetdomain,body=request).execute()
    last_call_len = len(initresponse['rows'])
    mapped_dictionary = map_response_to_dict(initresponse, data)
    while last_call_len == 25000:
        print('this is running')
        startRow = startRow + 25000
        request['startRow'] = startRow
        response = webmaster_search_analytics.searchanalytics().query(siteUrl=targetdomain,body=request).execute()
        last_call_len = len(response['rows'])
        mapped_dictionary = map_response_to_dict(response, data)
    return mapped_dictionary
        

In [14]:
def map_response_to_dict(response, data): # this turns the JSON response into an easy to manipulate dictionary for dataframes
    for i in response['rows']:
        date = i['keys'][0] #assumes that date is first
        data['date'].append(date)
        searchQuery = i['keys'][1] #assumes that query is second
        data['searchQuery'].append(searchQuery)
        page = i['keys'][2] #assumes that page is third
        data['page'].append(page)
        clicks = i['clicks']
        data['clicks'].append(clicks)
        impressions = i['impressions']
        data['impressions'].append(impressions)
        ctr = i['ctr']
        data['ctr'].append(ctr)
        position = i['position']
        data['position'].append(position)
    return data

In [15]:
big_dict = get_all_response(targetdomain,request) #pushing all data to a dictionary

this is running
this is running
this is running


In [16]:
seoDataFrame = pd.DataFrame(big_dict) #turning the dictionary into a PandaDataFrame

In [17]:
seoDataFrame.to_csv('/Users/oleksandrdagayev/code/jupyter_SEO/sample.csv') #set where the data is being written to